<a href="https://colab.research.google.com/github/graphlit/graphlit-samples/blob/main/python/Notebook%20Examples/Graphlit_2024_09_21_CrewAI_Medical_AI_Agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Description**

This example shows how to use Graphlit with CrewAI. Based on this [great blog post](https://farzzy.hashnode.dev/building-healthcare-ai-agent-medical-guidelines) and [notebook](https://farzzy.hashnode.dev/building-healthcare-ai-agent-medical-guidelines) written by Farzad Sunavala, we emulate Farzad's workflow and show how to use Graphlit instead of LlamaIndex, while already leveraging Azure AI Search under the hood.

**Requirements**

Prior to running this notebook, you will need to [signup](https://docs.graphlit.dev/getting-started/signup) for Graphlit, and [create a project](https://docs.graphlit.dev/getting-started/create-project).

You will need the Graphlit organization ID, preview environment ID and JWT secret from your created project.

Assign these properties as Colab secrets: GRAPHLIT_ORGANIZATION_ID, GRAPHLIT_ENVIRONMENT_ID and GRAPHLIT_JWT_SECRET.

For CrewAI, you will need your OpenAI API key.

Assign this property as Colab secret: OPENAI_API_KEY

---

In [1]:
!pip install --upgrade crewai

In [2]:
!pip install --upgrade graphlit-tools

  Using cached graphlit_tools-1.0.20240921020-py3-none-any.whl.metadata (2.4 kB)
  Using cached graphlit_client-1.0.20240914001-py3-none-any.whl.metadata (2.7 kB)
Using cached graphlit_tools-1.0.20240921020-py3-none-any.whl (14 kB)
Using cached graphlit_client-1.0.20240914001-py3-none-any.whl (197 kB)


Clone the original repo to access the PDFs

In [3]:
!rm -rf azure-ai-search-python-playground
!git clone https://github.com/farzad528/azure-ai-search-python-playground

Cloning into 'azure-ai-search-python-playground'...
remote: Enumerating objects: 244, done.
remote: Counting objects: 100% (82/82), done.
remote: Compressing objects: 100% (66/66), done.
remote: Total 244 (delta 21), reused 77 (delta 16), pack-reused 162 (from 1)
Receiving objects: 100% (244/244), 153.70 MiB | 17.62 MiB/s, done.
Resolving deltas: 100% (92/92), done.
Updating files: 100% (74/74), done.


Initialize Graphlit

In [4]:
import os
from google.colab import userdata
from graphlit import Graphlit
from graphlit_api import input_types, enums, exceptions

os.environ['GRAPHLIT_ORGANIZATION_ID'] = userdata.get('GRAPHLIT_ORGANIZATION_ID')
os.environ['GRAPHLIT_ENVIRONMENT_ID'] = userdata.get('GRAPHLIT_ENVIRONMENT_ID')
os.environ['GRAPHLIT_JWT_SECRET'] = userdata.get('GRAPHLIT_JWT_SECRET')

graphlit = Graphlit()

Initialize sample data

In [5]:
DATA_DIRECTORY = "azure-ai-search-python-playground"

PATH_PDFS = DATA_DIRECTORY + "/data/acc/"

Define Graphlit helper functions

In [6]:
import base64
import mimetypes
import asyncio
from typing import List, Optional
from tqdm.asyncio import tqdm

async def process_file(filename, progress_bar):
    pdf_path = os.path.join(PATH_PDFS, filename)

    content_id = await ingest_file(pdf_path)

    if content_id is not None:
        print(f'Ingested content [{content_id}] from [{pdf_path}].')

    progress_bar.update(1)

# NOTE: for local files, load from disk and convert to Base64 data
async def ingest_file(file_path: str):
    if graphlit.client is None:
        return;

    try:
        file_name = os.path.basename(file_path)
        content_name, _ = os.path.splitext(file_name)

        mime_type = mimetypes.guess_type(file_name)[0]

        if mime_type is None:
            print('Failed to infer MIME type')
            return None

        with open(file_path, "rb") as file:
            file_content = file.read()

            base64_content = base64.b64encode(file_content).decode('utf-8')

            # Using synchronous mode, so the notebook waits for the content to be ingested
            response = await graphlit.client.ingest_encoded_file(content_name, base64_content, mime_type, is_synchronous=True)

            return response.ingest_encoded_file.id if response.ingest_encoded_file is not None else None
    except exceptions.GraphQLClientError as e:
        print(str(e))
        return None

async def create_openai_specification(model: enums.OpenAIModels):
    if graphlit.client is None:
        return;

    input = input_types.SpecificationInput(
        name=f"OpenAI [{str(model)}]",
        type=enums.SpecificationTypes.COMPLETION,
        serviceType=enums.ModelServiceTypes.OPEN_AI,
        openAI=input_types.OpenAIModelPropertiesInput(
            model=model,
        ),
        strategy=input_types.ConversationStrategyInput(
            embedCitations=True
        ),
        retrievalStrategy=input_types.RetrievalStrategyInput(
            type=enums.RetrievalStrategyTypes.SECTION
        ),
        rerankingStrategy=input_types.RerankingStrategyInput(
            serviceType=enums.RerankingModelServiceTypes.COHERE
        )
    )

    try:
        response = await graphlit.client.create_specification(input)

        return response.create_specification.id if response.create_specification is not None else None
    except exceptions.GraphQLClientError as e:
        print(str(e))
        return None

    return None

async def create_conversation(specification_id: str):
    if graphlit.client is None:
        return;

    input = input_types.ConversationInput(
        name="Conversation",
        specification=input_types.EntityReferenceInput(
            id=specification_id
        )
    )

    try:
        response = await graphlit.client.create_conversation(input)

        return response.create_conversation.id if response.create_conversation is not None else None
    except exceptions.GraphQLClientError as e:
        print(str(e))
        return None

async def prompt_conversation(conversation_id: str, prompt: str):
    if graphlit.client is None:
        return None, None

    try:
        response = await graphlit.client.prompt_conversation(prompt, conversation_id)

        message = response.prompt_conversation.message.message if response.prompt_conversation is not None and response.prompt_conversation.message is not None else None
        citations = response.prompt_conversation.message.citations if response.prompt_conversation is not None and response.prompt_conversation.message is not None else None

        return message, citations
    except exceptions.GraphQLClientError as e:
        print(str(e))
        return None, None

# NOTE: these functions are just used to clean-up old data before executing the example
async def delete_all_specifications():
    if graphlit.client is None:
        return;

    _ = await graphlit.client.delete_all_specifications(is_synchronous=True)

async def delete_all_conversations():
    if graphlit.client is None:
        return;

    _ = await graphlit.client.delete_all_conversations(is_synchronous=True)

async def delete_all_contents():
    if graphlit.client is None:
        return;

    _ = await graphlit.client.delete_all_contents(is_synchronous=True)

Ingest sample PDFs

In [7]:
# Remove any existing contents; only needed for notebook example
await delete_all_contents()

print('Deleted all contents.')

# List of files to process
files = os.listdir(PATH_PDFS)

progress_bar = tqdm(total=len(files))

tasks = [process_file(filename, progress_bar) for filename in files]

await asyncio.gather(*tasks)

print('Ingested all contents.')

Deleted all contents.


  3%|▎         | 1/36 [00:19<11:11, 19.20s/it]

Ingested content [e96c97e6-0a1e-4832-81b7-6fbecdef1797] from [azure-ai-search-python-playground/data/acc/arnett-et-al-2014-aha-acc-hhs-strategies-to-enhance-application-of-clinical-practice-guidelines-in-patients-with.pdf].


  6%|▌         | 2/36 [00:20<04:46,  8.43s/it]

Ingested content [18f352c8-c1fe-48e5-aca2-de3eb56eb5f2] from [azure-ai-search-python-playground/data/acc/jacobs-et-al-2014-the-evolution-and-future-of-acc-aha-clinical-practice-guidelines-a-30-year-journey.pdf].
Ingested content [08d449be-bff4-4c7d-88e5-540d250d8b82] from [azure-ai-search-python-playground/data/acc/smith-et-al-2011-aha-accf-secondary-prevention-and-risk-reduction-therapy-for-patients-with-coronary-and-other.pdf].


...

Ingested content [9b650906-243b-4798-a145-4c9c5e0426cb] from [azure-ai-search-python-playground/data/acc/heidenreich-et-al-2022-2022-aha-acc-hfsa-guideline-for-the-management-of-heart-failure.pdf].


 97%|█████████▋| 35/36 [01:08<00:02,  2.24s/it]

Ingested content [91629542-19ff-4aeb-8ca9-a79152e7c5f3] from [azure-ai-search-python-playground/data/acc/al-khatib-et-al-2018-2017-aha-acc-hrs-guideline-for-management-of-patients-with-ventricular-arrhythmias-and-the.pdf].


100%|██████████| 36/36 [01:13<00:00,  2.91s/it]

Ingested content [d99818f6-7e83-4d30-bb4f-391e528a4f28] from [azure-ai-search-python-playground/data/acc/otto-et-al-2020-2020-acc-aha-guideline-for-the-management-of-patients-with-valvular-heart-disease.pdf].
Ingested all contents.


Copied from original notebook, for patient configuration

In [8]:
# Define complex patient profile
patient_profile = {
    "patient_id": "43454357890",
    "name": "Sarah Johnson",
    "age": 68,
    "gender": "Female",
    "height_cm": 165,
    "weight_kg": 72,
    "bmi": 26.4,
    "blood_type": "A-",
    "allergies": ["Sulfa drugs"],
    "current_medications": [
        "Flecainide",
        "Apixaban",
        "Metoprolol",
        "Atorvastatin",
        "Cilostazol",
    ],
    "chronic_conditions": {
        "atrial_fibrillation": True,
        "peripheral_arterial_disease": True,
        "hypertension": True,
        "hyperlipidemia": True,
        "coronary_artery_disease": False,
    },
    "family_medical_history": {"heart_disease": True, "stroke": True, "cancer": False},
    "lifestyle_factors": {
        "smoking": "Former smoker, quit 5 years ago",
        "alcohol_use": "Rare",
        "physical_activity_per_week": "4-5 days",
        "diet": "Heart-healthy, low-sodium",
        "sleep_hours_per_night": 6,
    },
    "recent_lab_results": {
        "inr": 1.4,
        "creatinine_mg_dl": 1.1,
        "ldl_cholesterol_mg_dl": 85,
        "hdl_cholesterol_mg_dl": 55,
        "blood_pressure_mm_hg": "135/80",
    },
    "vaccination_status": {
        "influenza_vaccine": True,
        "covid_vaccine": True,
        "pneumonia_vaccine": True,
    },
    "surgical_history": [
        "Carotid endarterectomy",
        "Cholecystectomy",
    ],
    "imaging_history": {
        "last_echocardiogram_date": "2024-07-15",
        "last_carotid_ultrasound_date": "2024-06-20",
    },
    "mental_health": {
        "anxiety": True,
        "depression": False,
        "cognitive_function_issues": False,
    },
    "preferences": {
        "preferred_treatment_type": [
            "Minimally invasive procedures",
            "Evidence-based treatments",
        ],
        "end_of_life_care": "Yes",
        "pain_management": "Non-opioid",
    },
    "recent_visits": [
        {
            "visit_date": "2024-08-25",
            "reason": "Follow-up for atrial fibrillation",
            "notes": "Recurrence of AF despite flecainide. Discussing ablation vs alternative antiarrhythmic options.",
        },
        {
            "visit_date": "2024-07-10",
            "reason": "Peripheral arterial disease management",
            "notes": "Stable claudication symptoms. Continue current management and exercise program.",
        },
    ],
    "af_management": {
        "current_treatment": "Flecainide",
        "treatment_history": [
            "Rate control with metoprolol",
            "Rhythm control with flecainide",
        ],
        "af_recurrence": True,
        "considering_options": ["Catheter ablation", "Alternative antiarrhythmic drug"],
        "chads2_vasc_score": 4,
        "has_bled_score": 2,
    },
    "pad_management": {
        "ankle_brachial_index": 0.78,
        "fontaine_classification": "Stage II",
        "current_treatment": ["Cilostazol", "Supervised exercise program"],
        "last_vascular_assessment": "2024-06-20",
    },
}

Execute example RAG prompt with citations

In [9]:
from IPython.display import display, Markdown, HTML
import time

# Remove any existing conversations and specifications; only needed for notebook example
await delete_all_conversations()
await delete_all_specifications()

print('Deleted all conversations and specifications.')

# Formulate a query based on Sarah Johnson's profile
prompt = (
    f"Which option reduces stroke risk for a {patient_profile['age']}-year-old "
    f"female with atrial fibrillation and peripheral arterial disease, "
    f"who is considering catheter ablation or antiarrhythmic drug therapy?"
)

model = enums.OpenAIModels.GPT4O_MINI_128K
#model = enums.OpenAIModels.O1_MINI_128K

specification_id = await create_openai_specification(model)

if specification_id is not None:
    print(f'Created specification [{specification_id}].')

    conversation_id = await create_conversation(specification_id=specification_id)

    if conversation_id is not None:
        message, citations = await prompt_conversation(conversation_id, prompt)

        if message is not None:
            display(Markdown(f'### Patient-Specific Question Response for {patient_profile["name"]}:'))
            display(Markdown(f'**Final Response:**\n{message}'))
            print()

        if citations is not None:
            print("\nReference Information:")

            for citation in citations:
                if citation is not None and citation.content is not None:
                    display(Markdown(f'**Citation [{citation.index}]:** {citation.content.name}'))
                    display(Markdown(citation.text))
                    print()


Deleted all conversations and specifications.
Created specification [57e7b740-95e8-4387-84eb-188c1f82f5d8].


### Patient-Specific Question Response for Sarah Johnson:

**Final Response:**
For a 68-year-old female with atrial fibrillation (AF) and peripheral arterial disease (PAD), the use of direct oral anticoagulants (DOACs) is recommended as a first-line option for stroke prevention. This recommendation is based on the patient's age and the presence of AF, which significantly increases her risk of thromboembolic events. Additionally, the CHA2DS2-VASc score, which assesses stroke risk, supports the need for anticoagulation in this demographic. [1][2][3][4][5]

While catheter ablation can be an effective treatment for AF, it is generally considered when antiarrhythmic drug therapy is ineffective or not preferred. In this case, the decision should also consider the patient's overall health status, the severity of her symptoms, and her preferences regarding treatment options. Antiarrhythmic drugs may help control AF symptoms, but they do not directly address stroke risk as effectively as anticoagulation. [6][7][8][9][10]





Reference Information:


**Citation [1]:** heidenreich-et-al-2022-2022-aha-acc-hfsa-guideline-for-the-management-of-heart-failure

...

9.8. Atrial Fibrillation
S9.8-1. Healey JS, Baranchuk A, Crystal E, et al. Prevention of atrial fibrillation with angiotensin- converting enzyme inhibitors and angiotensin recep- tor blockers: a meta-analysis. J Am Coll Cardiol. 2005; 45:1832-9.
S9.8-2. Zhao D, Wang Z-M, Wang L-S. Prevention of atrial fibrillation with renin-angiotensin system in- hibitors on essential hypertensive patients: a meta- analysis of randomized controlled trials. J Biomed Res. 2015;29:475-85.
S9.8-3. Kistler PM, Sanders P, Fynn SP, et al. Elec- trophysiologic and electroanatomic changes in the human atrium associated with age. J Am Coll Cardiol. 2004;44:109-16.
S9.8-4. January CT, Wann LS, Alpert JS, et al. 2014 AHA/ACC/HRS guideline for the management of pa- tients with atrial fibrillation: a report of the American College of Cardiology/American Heart Association Task Force on Practice Guidelines and the Heart Rhythm Society. J Am Coll Cardiol. 2014;64:e1-76.
59.8-5. Olesen JB, Torp-Pedersen C, Hansen ML, et al. The value of the CHA2DS2-VASc score for refining stroke risk stratification in patients with atrial fibrilla- tion with a CHADS2 score 0-1: a nationwide cohort study. Thromb Haemost. 2012;107:1172-9.
S9.8-6. Healey JS, Connolly SJ. Atrial fibrillation: hypertension as a causative agent, risk factor for
complications, and potential therapeutic target. Am J Cardiol. 2003;91:9G-14G.
S9.8-7. Hansson L, Lindholm LH, Ekbom T, et al. Randomised trial of old and new antihypertensive drugs in elderly patients: cardiovascular mortality and morbidity. The Swedish Trial in Old Patients with Hypertension-2 study. Lancet. 1999;354:1751-6.
S9.8-8. Julius S, Kjeldsen SE, Weber M, et al. Outcomes in hypertensive patients at high cardiovas- cular risk treated with regimens based on valsartan or amlodipine: the VALUE randomised trial. Lancet. 2004;363:2022-31.
S9.8-9. Wachtell K, Lehto M, Gerdts E, et al. Angio- tensin II receptor blockade reduces new-onset atrial fibrillation and subsequent stroke compared to aten- olol: the Losartan Intervention For End Point Reduc- tion in Hypertension (LIFE) study. J Am Coll Cardiol. 2005;45:712-9.
59.8-10. Haywood LJ, Ford CE, Crow RS, et al. Atrial fibrillation at baseline and during follow-up in ALLHAT (Antihypertensive and Lipid-Lowering Treatment to Prevent Heart Attack Trial). J Am Coll Cardiol. 2009; 54:2023-31.
59.8-11. Hansson L, Lindholm LH, Niskanen L, et al. Effect of angiotensin-converting-enzyme inhibition compared with conventional therapy on cardiovascular morbidity and mortality in hypertension: the Captopril Prevention Project (CAPPP) randomised trial. Lancet. 1999;353:611-6.

**Citation [9]:** ommen-et-al-2024-2024-aha-acc-amssm-hrs-paces-scmr-guideline-for-the-management-of-hypertrophic-cardiomyopathy

TOP 10 TAKE-HOME MESSAGES
1. Shared decision-making is essential to provide the best clinical care. This involves thoughtful dialogue among patients, families, and their care team in which health care professionals present all available testing and treatment options; discuss the risks, benefits, and applicability of those options to the individual pa- tient; and ensure the patient expresses their personal preferences and goals to develop their treatment plan.
2. Although the primary cardiology team can initiate evaluation, treatment, and longitudinal care, referral to multidisciplinary hypertrophic cardiomyopathy (HCM) centers with appropriate expertise can be important to optimizing care for patients with HCM. Challenging treatment decisions-where reasonable alternatives exist, where the strength of recommendation is weak (eg, any decision relying on a Class of Recommendation 2b) or is particularly nuanced (eg, interpretation of genetic testing; primary prevention implantable cardioverter-defibrillator decision-making), and for HCM-specific invasive procedures-may critically benefit from involving specialized HCM centers.
3. Careful ascertainment of family history, counseling patients with HCM about the potential for genetic transmission of HCM, and options for genetic testing are cornerstones of care. Screening first-degree family members of patients with HCM, using either genetic testing, serial imaging, or electrocardiogramaur- veillance as appropriate, can begin at any age and can be influenced by specifics of the patient and family history and family preference. Because screening recommendations for family members hinge on the pathogenicity of any detected variants, the reported pathogenicity should be reconfirmed every 2 to 3 years, and input from specialized HCM centers with genetics expertise may be valuable.
4. Assessing a patient's risk for sudden cardiac death is an important component of management. Integrating the presence or absence of established risk markers with tools to estimate individual risk score will facilitate the patient's ability to participate in decision-making regarding implantable cardioverter- defibrillator placement. These discussions should incorporate a patient's personal level of risk tolerance and their specific treatment goals.
5. The risk factors for sudden cardiac death in children with HCM carry different weights and components than those used in adult patients. Pediatric risk stratification also varies with age and must account for different body sizes. Coupled with the complexity of placing implantable cardioverter-defibrillators in young patients with anticipated growth and a higher risk of device complications, the threshold for
implantable cardioverter-defibrillator implantation in children often differs from adults. These differences are best addressed at comprehensive HCM centers with expertise in caring for children with HCM. New risk calculators, specific to children and adolescents, have been validated and can help young patients and their families contextualize their estimated risk of sudden cardiac death.
6. Cardiac myosin inhibitors are now available to treat patients with symptomatic obstructive HCM. This new class of medication inhibits actin-myosin interaction, thus decreasing cardiac contractility and reducing left ventricular outflow tract obstruction. Mava- camten is currently the only U.S. Food and Drug Administration-approved agent. These agents can be beneficial for patients with obstructive HCM who do not derive adequate symptomatic relief from first-line drug therapy.
7. Invasive septal reduction therapies (surgical septal myectomy and alcohol septal ablation), when per- formed by experienced HCM teams at dedicated cen- ters, can provide safe and effective symptomatic relief for patients with drug-refractory or severe outflow tract obstruction. Given the data on the significantly improved outcomes at comprehensive HCM centers, these decisions represent an optimal opportunity for referral.
8. Patients with HCM and persistent or paroxysmal atrial fibrillation have a sufficiently increased risk of stroke such that oral anticoagulation with direct-acting oral anticoagulants (or alternatively warfarin) should be considered the default treatment option irrespective of the CHA2DS2-VASc score. New tools to stratify risk for incident atrial fibrillation have been developed and may assist in determining the frequency of screening patients with ambulatory telemetry. Because rapid atrial fibrillation is often poorly toler- ated in patients with HCM, maintenance of sinus rhythm and rate control are key treatment goals.
9. Exercise stress testing is particularly helpful in determining overall exercise tolerance and for latent exercise provoked left ventricular outflow tract obstruction. Because children may not describe symptoms readily, routine exercise testing can be particularly important for young patients.
10. Increasingly, data affirm that the beneficial effects of exercise on general health are extended to patients with HCM. Healthy recreational exercise (light [<3 metabolic equivalents], moderate [3-6 metabolic equivalents], and vigorous [>6 metabolic equivalents] intensity levels) has not been associated with increased risk of ventricular arrhythmia events in short-term studies. If patients pursue rigorous exer- cise training for the purpose of performance or
competition, it is important to engage in a compre- hensive discussion and seek input from expert HCM professionals regarding the potential risks and bene- fits, to develop an individualized training plan, and to establish a regular schedule for reevaluation.

**Citation [10]:** ommen-et-al-2024-2024-aha-acc-amssm-hrs-paces-scmr-guideline-for-the-management-of-hypertrophic-cardiomyopathy

8.5. Management of Patients With HCM and Ventricular Arrhythmias
Recommendations for the Management of Patients With HCM and Ventricular Arrhythmias Referenced studies that support the recommendations are summarized in the Online Data Supplement.
| COR | LOE | RECOMMENDATIONS |
| --- | --- | --- |
| 1 | B-NR | 1. In patients with HCM and recurrent, poorly tolerated life-threatening ventricular tachyarrhythmias re- fractory to maximal antiarrhythmic drug therapy and ablation, heart transplantation assessment is indi- cated in accordance with current listing criteria.1,2 |
| 1 | B-NR* C-LD+ | 2. In adults with HCM and symptomatic ventricular arrhythmias or recurrent ICD shocks despite beta-blocker use, antiarrhythmic drug therapy (eg, amiodarone,* dofetilide, t mexiletine, t or sotalolt) is recommended, with the choice of agent guided by age, underlying comorbidities, severity of disease, patient preferences, and balance between efficacy and safety.3-6 |
| 1 | C-LD | 3. In children with HCM and recurrent ventricular arrhythmias despite beta-blocker use, antiarrhythmic drug therapy (eg, amiodarone,3,4 mexiletine,6 sotalol3,4) is recommended, with the choice of agent guided by age, underlying comorbidities, severity of disease, patient preferences, and balance of efficacy and safety. |
| 1 | C-LD | 4. In patients with HCM and pacing-capable ICDs, programming antitachycardia pacing is recommended to minimize risk of shocks.7,8 |
| 2a | C-LD | 5. In patients with HCM and recurrent symptomatic sustained monomorphic VT, or recurrent ICD shocks despite optimal device programming, and in whom antiarrhythmic drug therapy is either ineffective, not tolerated, or not preferred, catheter ablation can be useful for reducing arrhythmia burden.9-11 |

*Indicates the LOE for amiodarone. tIndicates the LOE for dofetilide, mexiletine, or sotalol.

Initialize CrewAI

In [10]:
from crewai import Agent, Task, Crew, Process
from graphlit_tools import PromptTool
from textwrap import dedent

os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY')

prompt_tool = PromptTool(graphlit)

# Embed patient profile information into the agent goals
# Define the CrewAI agents
guideline_expert = Agent(
    role="Guideline Expert",
    goal=(
        f"Retrieve and summarize relevant ACC guidelines for a {patient_profile['age']}-year-old "
        f"with atrial fibrillation and peripheral arterial disease. "
        "Focus on treatment options such as catheter ablation and antiarrhythmic therapy."
    ),
    backstory="You are an expert on ACC guidelines for managing atrial fibrillation and PAD.",
    tools=[prompt_tool],
    verbose=True,
)

patient_educator = Agent(
    role="Patient Educator",
    goal=(
        f"Translate the medical guidelines into easy-to-understand terms for Sarah Johnson. "
        "Focus on explaining the treatment options available for managing atrial fibrillation."
    ),
    backstory="You explain complex medical terms in patient-friendly language.",
    verbose=True,
)

treatment_planner = Agent(
    role="Treatment Planner",
    goal=(
        f"Create a personalized treatment plan for Sarah Johnson, considering her preference for minimally invasive procedures "
        f"and her options of catheter ablation or antiarrhythmic therapy."
    ),
    backstory="You specialize in personalized treatment plans based on patient history and preferences.",
    verbose=True,
)

output_generator = Agent(
    role="Output Generator",
    goal=(
        f"Compile the information into a comprehensive patient decision aid document for Sarah Johnson, "
        f"including a clear summary of her treatment options and next steps."
    ),
    backstory="You ensure that the medical recommendations are presented clearly and concisely.",
    verbose=True,
)

# Define tasks for each agent
task1 = Task(
    description="Retrieve ACC guidelines for managing atrial fibrillation and PAD.",
    expected_output="Summarized guidelines with a focus on catheter ablation and stroke risk reduction.",
    agent=guideline_expert,
)

task2 = Task(
    description="Translate the guidelines into patient-friendly language.",
    expected_output="Simplified, patient-friendly explanations of the treatment options.",
    agent=patient_educator,
)

task3 = Task(
    description="Personalize the treatment plan according to the patient's preferences.",
    expected_output="A treatment plan tailored to the patient's specific conditions and preferences.",
    agent=treatment_planner,
)

task4 = Task(
    description="Generate a patient decision aid document.",
    expected_output="A final decision aid document summarizing the patient's condition and treatment recommendations.",
    agent=output_generator,
)

# Create the Crew and define the process flow
crew = Crew(
    agents=[guideline_expert, patient_educator, treatment_planner, output_generator],
    tasks=[task1, task2, task3, task4],
    process=Process.sequential,  # Ensures tasks are executed in sequence
    verbose=True
)

# Execute the multi-step reasoning process
result = await crew.kickoff_async()

# Display the final patient decision aid
print("Final Patient Decision Aid:")
print(result)

# Agent: Guideline Expert
## Task: Retrieve ACC guidelines for managing atrial fibrillation and PAD.


# Agent: Guideline Expert
## Thought: Thought: I need to retrieve the relevant ACC guidelines for managing atrial fibrillation and peripheral arterial disease (PAD), focusing on treatment options such as catheter ablation and stroke risk reduction.
## Using tool: Graphlit prompt tool
## Tool Input: 
{
  "prompt": "Retrieve ACC guidelines for managing atrial fibrillation and peripheral arterial disease with a focus on catheter ablation and stroke risk reduction."
}
## Tool Output: 
The 2014 AHA/ACC/HRS guideline for the management of patients with atrial fibrillation includes recommendations on catheter ablation and stroke risk reduction. Catheter ablation is recommended for patients with symptomatic atrial fibrillation who have failed antiarrhythmic drug therapy. Stroke risk reduction is primarily managed through anticoagulation therapy based on the patient's risk factors.

The 2019 A